# Full Adder
A full adder is a digital circuit that performs the operation of addition of two given binary numbers.
We can use signed or unsigned numbers but for this circuit we will use the simpler unsigned representation.
So, for 3 bits we can encode 8 different numbers:

|Unsigned Binary|Decimal|
|:-:|:-:|
|$000$|$0$|
|$001$|$1$|
|$010$|$2$|
|$011$|$3$|
|$100$|$4$|
|$101$|$5$|
|$110$|$6$|
|$111$|$7$|

We will implement a 3-qubit full adder circuit that can compute the addition of two three-qubit numbers and an optional carry in qubit.

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

We will need two quantum registers to store the unsigned binary representations of out two numbers $p$ and $q$.  
We also declare $nbits$, a variable that dictates that we use three qubits to represent our numbers. 

In [ ]:
nbits = 3
p = QuantumRegister(nbits, 'p')
q = QuantumRegister(nbits, 'q')

We also need an additional quantum register to store the initial and the subsequent carry qubits from each iteration of the addition $c_{in}$(`cin`)
and a scratch register $o$(`o`) that will contain the carry out qubit. `o := 0`.

In [ ]:
cin = QuantumRegister(nbits, 'cin')
o = QuantumRegister(1, 'o')

At last we declare a classic register $o'$(`oprime`) that will contain the measured output of the full adder.

In [ ]:
oprime = ClassicalRegister(nbits, "o'")

We finally declare a quantum circuit (`circ`) that contains all the quantum and classical registers from above.

In [ ]:
circ = QuantumCircuit(p, q, cin, o, oprime)

In [ ]:
decp = input('Input a decimal number "p" (0 <= x < 8): ')
i = 0
for digit in reversed(bin(int(decp)).removeprefix('0b')):
    if digit == '1':
        circ.x(p[i])
    i += 1

decq = input('Input a decimal number "q" (0 <= x < 8): ')
i = 0
for digit in reversed(bin(int(decq)).removeprefix('0b')):
    if digit == '1':
        circ.x(q[i])
    i += 1
circ.barrier()

In [ ]:
for i in range(nbits):
    circ.ccx(p[i], q[i], o)
    circ.cx(p[i], q[i])
    circ.ccx(q[i], cin[i], o)
    circ.cx(q[i], cin[i])
    circ.cx(p[i], q[i])
    if i+1 < nbits:
        circ.swap(o, cin[i+1])
    circ.barrier()
circ.measure(cin, oprime)

In [ ]:
from qiskit.visualization import circuit_drawer
circuit_drawer(circ, output='mpl', fold=-1)

In [ ]:
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram

In [ ]:
backend = Aer.get_backend('qasm_simulator')
result = backend.run(circ).result()

In [ ]:
decres = list(result.get_counts().int_outcomes().keys())[0]
print(f'{decp} + {decq} = {decres}')
plot_histogram(result.get_counts())